In [2]:
import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

import matplotlib.pyplot as plt
import numpy as np

import os
import json

from torch.utils.data import DataLoader, Dataset
from PIL import Image

from torchinfo import summary
import torch.optim as optim
from tqdm.notebook import tqdm
from tensorboardX import SummaryWriter
import glob

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [4]:
# Training settings
batch_size = 2
lr = 3e-5
gamma = 0.7
seed = 42
image_size = (3,256,256)
batch_size = 2
sequence_length = 20

# DATASET

video - frame(초당 1프레임 불러들이기) [100초일 경우 100프레임] - 프레임 20개를 1개의 sequence로 바꾸기

In [5]:
# import random
# seed = 777
# random.seed(seed)
# example = torch.randn((16,3,224,224))
# y = [random.randrange(0,2) for i in range(16)]
# print(example.shape)
# print(y)

In [6]:
os.makedirs('data', exist_ok=True)

In [7]:
train_dir = 'data/train'
test_dir = 'data/test'

In [8]:
train_list = glob.glob(os.path.join(train_dir,'*.jpg'))
test_list = glob.glob(os.path.join(test_dir, '*.jpg'))

In [9]:
print(f"Train Data: {len(train_list)}")
print(f"Test Data: {len(test_list)}")

Train Data: 25000
Test Data: 12500


In [10]:
labels = [path.split('/')[-1].split('.')[0] for path in train_list]

In [ ]:
random_idx = np.random.randint(1, len(train_list), size=9)
fig, axes = plt.subplots(3, 3, figsize=(16, 12))

for idx, ax in enumerate(axes.ravel()):
    img = Image.open(train_list[idx])
    ax.set_title(labels[idx])
    ax.imshow(img)


# split

In [ ]:
train_list, valid_list = train_test_split(train_list, 
                                          test_size=0.2,
                                          stratify=labels,
                                          random_state=seed)

In [1]:
print(f"Train Data: {len(train_list)}")
print(f"Validation Data: {len(valid_list)}")
print(f"Test Data: {len(test_list)}")

NameError: name 'train_list' is not defined

In [ ]:
def MakeSequence(x,y,sequence_length):
    x_seq = []
    y_seq = []
    for i in range(sequence_length):
        x_seq.append(x[i]

In [48]:
class SeqDataset(Dataset):
    def __init__(self,file_list,transform=None):
        self.file_list = file_list
        sefl.transform = transform
        
    def __len__(self):
        sefl.filelength = len(self.file_list)
        return self.filelength
    
    def __getitem__(self,idx):
        
        x_seq = []
        y_seq = []
        for i in range(sequence_length):    
            img_path = self.file_list[idx]
            img = Image.open(img_path)
            img_transformed = self.trasnform(img)
            x_seq.append(img_transformed)
            
            label = open(label,'r')
            label = label.readline()
            label = 1 if label == "1" else 0
            y_seq.append(label)
            
        return x_seq,y_seq
    
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    
])


In [6]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=None, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,
                                          shuffle=True)
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=None, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=32,
                                         shuffle=False)

In [7]:
class VGG19(nn.Module):
    def __init__(self):
        super(VGG19, self).__init__()
        self.conv = nn.Sequential(
            #3 224 128
            nn.Conv2d(3, 64, 3, padding=1),nn.LeakyReLU(0.2),
            nn.Conv2d(64, 64, 3, padding=1),nn.LeakyReLU(0.2),
            nn.MaxPool2d(2, 2),
            #64 112 64
            nn.Conv2d(64, 128, 3, padding=1),nn.LeakyReLU(0.2),
            nn.Conv2d(128, 128, 3, padding=1),nn.LeakyReLU(0.2),
            nn.MaxPool2d(2, 2),
            #128 56 32
            nn.Conv2d(128, 256, 3, padding=1),nn.LeakyReLU(0.2),
            nn.Conv2d(256, 256, 3, padding=1),nn.LeakyReLU(0.2),
            nn.Conv2d(256, 256, 3, padding=1),nn.LeakyReLU(0.2),
            nn.MaxPool2d(2, 2),
            #256 28 16
            nn.Conv2d(256, 512, 3, padding=1),nn.LeakyReLU(0.2),
            nn.Conv2d(512, 512, 3, padding=1),nn.LeakyReLU(0.2),
            nn.Conv2d(512, 512, 3, padding=1),nn.LeakyReLU(0.2),
            nn.MaxPool2d(2, 2),
            #512 14 8
            #for memory
#             nn.Conv2d(512, 512, 3, padding=1),nn.LeakyReLU(0.2),
#             nn.Conv2d(512, 512, 3, padding=1),nn.LeakyReLU(0.2),
#             nn.Conv2d(512, 512, 3, padding=1),nn.LeakyReLU(0.2),
#             nn.MaxPool2d(2, 2)
        )
        #512 7 4
        self.avg_pool = nn.AvgPool2d(14)
        #512 1 1
        self.classifier = nn.Linear(512, 10)

    def forward(self, x):
        features = self.conv(x)
        x = self.avg_pool(features)
        x = torch.squeeze(x)
        
        #print(avg_pool.size())
#         x = x.view(features.size(0), -1)
        #print(flatten.size())
#         x = self.classifier(x)
        #x = self.softmax(x)
    
        return x

batch, sequence, channel, w, h

2,3,3,4,4

x = torch.FloatTensor([[[[[1,1,1,1],[1,1,1,1],[1,1,1,1],[1,1,1,1]],
                     [[1,1,1,1],[1,1,1,1],[1,1,1,1],[1,1,1,1]],
                     [[1,1,1,1],[1,1,1,1],[1,1,1,1],[1,1,1,1]]],
                     [[[2,2,2,2],[2,2,2,2],[2,2,2,2],[2,2,2,2]],
                     [[2,2,2,2],[2,2,2,2],[2,2,2,2],[2,2,2,2]],
                     [[2,2,2,2],[2,2,2,2],[2,2,2,2],[2,2,2,2]]],
                      [[[3,3,3,3],[3,3,3,3],[3,3,3,3],[3,3,3,3]],
                     [[3,3,3,3],[3,3,3,3],[3,3,3,3],[3,3,3,3]],
                     [[3,3,3,3],[3,3,3,3],[3,3,3,3],[3,3,3,3]]]],
                      [[[[4,4,4,4],[4,4,4,4],[4,4,4,4],[4,4,4,4]],
                     [[4,4,4,4],[4,4,4,4],[4,4,4,4],[4,4,4,4]],
                     [[4,4,4,4],[4,4,4,4],[4,4,4,4],[4,4,4,4]]],
                     [[[5,5,5,5],[5,5,5,5],[5,5,5,5],[5,5,5,5]],
                     [[5,5,5,5],[5,5,5,5],[5,5,5,5],[5,5,5,5]],
                     [[5,5,5,5],[5,5,5,5],[5,5,5,5],[5,5,5,5]]],
                      [[[6,6,6,6],[6,6,6,6],[6,6,6,6],[6,6,6,6]],
                     [[6,6,6,6],[6,6,6,6],[6,6,6,6],[6,6,6,6]],
                     [[6,6,6,6],[6,6,6,6],[6,6,6,6],[6,6,6,6]]]]]
                     )
                     
#torch.Size([2, 3, 3, 4, 4])

print(x.shape)

x.view(6,3,4,4)

print(x)

In [7]:
import torch
import torch.nn as nn
import torchvision.models as models
from torch.nn.utils.rnn import pack_padded_sequence
import torch.nn.functional as F
from torchvision.models import resnet18, resnet101


class CNNLSTM(nn.Module):
    def __init__(self, num_classes=2):
        super(CNNLSTM, self).__init__()
        self.resnet = resnet101(pretrained=True)
        self.resnet.fc = nn.Sequential(nn.Linear(self.resnet.fc.in_features, 300))
        self.lstm = nn.LSTM(input_size=300, hidden_size=256, num_layers=3)
        self.fc1 = nn.Linear(256, 128)
        self.fc2 = nn.Linear(128, num_classes)
       
    def forward(self, x_3d):
        hidden = None
        for t in range(x_3d.size(1)):
            with torch.no_grad():
                x = self.resnet(x_3d[:, t, :, :, :])  
            out, hidden = self.lstm(x.unsqueeze(0), hidden)         

        x = self.fc1(out[-1, :, :])
        x = F.relu(x)
        x = self.fc2(x)
        return x

In [ ]:
model = CNNLSTM().to(device)

In [8]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

class CNNLSTM(nn.Module):
    def __init__(self,hidden_dim=128):
        super().__init__()
        
        ## embedding with cnn
        self.cnn = VGG19()
        self.hidden_dim = hidden_dim
        self.lstm = nn.LSTM(512,
                            hidden_dim,
                            batch_first = True
                           )
        self.drop = nn.Dropout(p=0.5)
        self.fc = nn.Linear(hidden_dim,1)
        
        
        
        
    def forward(self,x):
        # num_layers*bidirectional, batch_size, n_hidden)
        hidden = torch.zeros(1, 2, 128, requires_grad=True).to(device)
        cell = torch.zeros(1, 2, 128, requires_grad=True).to(device)
        
        batchsize, time_steps,C,H,W = x.size()
        c_in = x.view(batchsize * time_steps,C,H,W)
        c_out = self.cnn(c_in)
        r_in = c_out.view(batchsize, time_steps, -1)
        r_out,(hn,cn) = self.lstm(r_in,(hidden,cell))
        result = self.drop(r_out)
        result = self.fc(result[-1])
        result = torch.squeeze(result)
        return torch.sigmoid(result)

In [9]:
model = CNNLSTM().to(device)

In [10]:
summary(model,(batch_size,sequence_length,3,224,224))

C:\ProgramData\Anaconda3\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Layer (type:depth-idx)                   Output Shape              Param #
CNNLSTM                                  --                        --
├─VGG19: 1-1                             [40, 512]                 --
│    └─Sequential: 2-1                   [40, 512, 14, 14]         --
│    │    └─Conv2d: 3-1                  [40, 64, 224, 224]        1,792
│    │    └─LeakyReLU: 3-2               [40, 64, 224, 224]        --
│    │    └─Conv2d: 3-3                  [40, 64, 224, 224]        36,928
│    │    └─LeakyReLU: 3-4               [40, 64, 224, 224]        --
│    │    └─MaxPool2d: 3-5               [40, 64, 112, 112]        --
│    │    └─Conv2d: 3-6                  [40, 128, 112, 112]       73,856
│    │    └─LeakyReLU: 3-7               [40, 128, 112, 112]       --
│    │    └─Conv2d: 3-8                  [40, 128, 112, 112]       147,584
│    │    └─LeakyReLU: 3-9               [40, 128, 112, 112]       --
│    │    └─MaxPool2d: 3-10              [40, 128, 56, 56]         --

In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=0.00001)

In [13]:
writer= SummaryWriter('runs/cnn-lstm')

In [14]:
# for epoch in range(10):
#     epoch_loss = 0
#     epoch_acc = 0
    
#     for data,label in tqdm(trainloader):
#         data = data.to(device)
#         label = label.to(device)
        
#         output = model(data)
        
#         loss = criterion(output,label)
        
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
        
#         acc = (output.argmax(dim=1) == label).float().mean()
#         epoch_loss += loss / len(trainloader)
#         epoch_acc += acc /len(trainloader)
        
#     with torch.no_grad():
#         epoch_val_accuracy = 0
#         epoch_val_loss = 0
#         for data, label in testloader:
#             data = data.to(device)
#             label = label.to(device)

#             val_output = model(data)
#             val_loss = criterion(val_output, label)

#             acc = (val_output.argmax(dim=1) == label).float().mean()
#             epoch_val_accuracy += acc / len(testloader)
#             epoch_val_loss += val_loss / len(testloader)
#         writer.add_scalar('validation_loss',epoch_val_loss,epoch)

#     writer.add_scalar('training_loss',epoch_loss,epoch)


#     print(
#         f"Epoch : {epoch+1} - loss : {epoch_loss:.4f} - acc: {epoch_acc:.4f} - val_loss : {epoch_val_loss:.4f} - val_acc: {epoch_val_accuracy:.4f}\n"
#     )
        
        